<a href="https://colab.research.google.com/github/JoseAndresEscobar19/Materia_integradora2020I/blob/master/AnalisisCBIS_DDSM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Importando las librerias

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
rutadrive="/content/drive/My Drive/Materia_integradora/Datasets/CBIS-DDMS/"
ruta="/content/Dataset/"

In [81]:
# !rm *.nii.gz
# !rm -r Dataset
# !rm -r ddsm_tools/

In [ ]:
!cp -r /content/drive/My\ Drive/Materia_integradora/Datasets/CBIS-DDMS/Librerias/ddsm_tools /content/ddsm_tools
!chmod 755 ddsm_tools/jpegdir/jpeg
!apt-get install -y pigz
!apt-get install dcm2niix
!pip install pydicom
!pip install opencv-python
!pip install pillow
!pip install nipype
!pip install nibabel
 
import ddsm_tools.my_ddsm_converter as ddsm
import pandas as pd 
import os
import zipfile
import numpy as np
import urllib.request, urllib.error, urllib.parse
import urllib.request, urllib.parse, urllib.error
import sys
import math
import tempfile
import pydicom
from pydicom.uid import ImplicitVRLittleEndian
from pydicom.dataset import Dataset, FileDataset
import matplotlib.pyplot as plt
import cv2
import csv
import shutil
import subprocess
import glob
import nibabel as nib
from PIL import Image

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-440
Use 'apt autoremove' to remove it.
The following NEW packages will be installed:
  pigz
0 upgraded, 1 newly installed, 0 to remove and 35 not upgraded.
Need to get 57.4 kB of archives.
After this operation, 259 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 pigz amd64 2.4-1 [57.4 kB]
Fetched 57.4 kB in 1s (64.1 kB/s)
Selecting previously unselected package pigz.
(Reading database ... 144465 files and directories currently installed.)
Preparing to unpack .../archives/pigz_2.4-1_amd64.deb ...
Unpacking pigz (2.4-1) ...
Setting up pigz (2.4-1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically install

In [82]:
if not os.path.exists("Dataset"):
  os.mkdir("Dataset")

### Obteniendo estadisticas del dataset

In [ ]:
df_calc_train=pd.read_csv(rutadrive+"calc_case_description_train_set.csv")
df_calc_train2=df_calc_train[df_calc_train['pathology'].isin(["MALIGNANT","BENIGN"])]
df_2.head()
print(df_2["pathology"].value_counts())
dff_2 = df_2[["pathology","patient_id"]]
d=dff_2.groupby(['patient_id','pathology']).count().reset_index()
d["pathology"].value_counts()

MALIGNANT    544
BENIGN       528
Name: pathology, dtype: int64


MALIGNANT    276
BENIGN       258
Name: pathology, dtype: int64

In [ ]:
df_calc_test=pd.read_csv(rutadrive+"calc_case_description_test_set.csv")
df_calc_test2=df_calc_test[df_calc_test['pathology'].isin(["MALIGNANT","BENIGN"])]
df_calc_test2.head()
print(df_calc_test2["pathology"].value_counts())
dff_2 = df_calc_test2[["pathology","patient_id"]]
d=dff_2.groupby(['patient_id','pathology']).count().reset_index()
d["pathology"].value_counts()

BENIGN       130
MALIGNANT    129
Name: pathology, dtype: int64


BENIGN       70
MALIGNANT    67
Name: pathology, dtype: int64

In [ ]:
df_masa_train=pd.read_csv(rutadrive+"mass_case_description_train_set.csv")
df_masa_train2=df_masa_train[df_masa_train['pathology'].isin(["MALIGNANT","BENIGN"])]
df_masa_train2.head()
print(df_masa_train2["pathology"].value_counts())
dff_2 = df_masa_train2[["pathology","patient_id"]]
d=dff_2.groupby(['patient_id','pathology']).count().reset_index()
d["pathology"].value_counts()

MALIGNANT    637
BENIGN       577
Name: pathology, dtype: int64


MALIGNANT    346
BENIGN       304
Name: pathology, dtype: int64

In [ ]:
df_masa_test=pd.read_csv(rutadrive+"mass_case_description_test_set.csv")
df_masa_test2=df_masa_test[df_masa_test['pathology'].isin(["MALIGNANT","BENIGN"])]
df_masa_test2.head()
print(df_masa_test2["pathology"].value_counts())
dff_2 = df_masa_test2[["pathology","patient_id"]]
d=dff_2.groupby(['patient_id','pathology']).count().reset_index()
d["pathology"].value_counts()

BENIGN       194
MALIGNANT    147
Name: pathology, dtype: int64


BENIGN       101
MALIGNANT     85
Name: pathology, dtype: int64

### Creando particiones de los datasets

In [ ]:
df_benigno_calc_full=df_calc_train[df_calc_train['pathology'].isin(["BENIGN"])]
df_benigno_calc = df_benigno_calc_full.head(50)
df_maligno_calc_full=df_calc_train[df_calc_train['pathology'].isin(["MALIGNANT"])]
df_maligno_calc = df_maligno_calc_full.head(50)
df_benigno_mass_full=df_masa_train[df_masa_train['pathology'].isin(["BENIGN"])]
df_benigno_mass = df_benigno_mass_full.head(50)
df_maligno_mass_full=df_masa_train[df_masa_train['pathology'].isin(["MALIGNANT"])]
df_maligno_mass = df_maligno_mass_full.head(50)

### Clase para conexion y descarga de imagenes

In [ ]:
#
# Refer https://wiki.cancerimagingarchive.net/display/Public/REST+API+Usage+Guide for complete list of API
#
class TCIAClient:
    GET_IMAGE = "getImage"
 
    def __init__(self , baseUrl, resource):
        self.baseUrl = baseUrl + "/" + resource
 
    def execute(self, url, queryParameters={}):
        queryParameters = dict((k, v) for k, v in queryParameters.items() if v)
        queryString = "?%s" % urllib.parse.urlencode(queryParameters)
        requestUrl = url + queryString
        request = urllib.request.Request(url=requestUrl)
        resp = urllib.request.urlopen(request)
        return resp
 
 
    def get_image(self , seriesInstanceUid , downloadPath, zipFileName):
        serviceUrl = self.baseUrl + "/query/" + self.GET_IMAGE
        queryParameters = { "SeriesInstanceUID" : seriesInstanceUid }
        os.umask(0o002)
        try:
            file = os.path.join(downloadPath, zipFileName)
            resp = self.execute( serviceUrl , queryParameters)
            downloaded = 0
            CHUNK = 256 * 10240
            with open(file, 'wb') as fp:
                while True:
                    chunk = resp.read(CHUNK)
                    downloaded += len(chunk)
                    if not chunk: break
                    fp.write(chunk)
        except urllib.error.HTTPError as e:
            print("HTTP Error:",e.code , serviceUrl)
            return False
        except urllib.error.URLError as e:
            print("URL Error:",e.reason , serviceUrl)
            return False
 
        return True

In [ ]:
tcia_client = TCIAClient(baseUrl="https://services.cancerimagingarchive.net/services/v4",resource = "TCIA")

## Descargando las imagenes y convirtiendolas a nii

In [84]:
def descargar_imagenes_imgs_2(tcia,data):
  datos={"patient_id":[],"patient_name":[],"abnormality type":[],"breast_density":[],
         "subtlety":[],"assessment":[],"image_path":[],"roi_path":[],"pathology":[]}
  for p_id,imgpath,roipath,pathology,abnormality,density,subtlety,assessment in zip( data["patient_id"],
                                                                                                data["image file path"],
                                                                                            data["ROI mask file path"],
                                                                                            data["pathology"],
                                                                                            data["abnormality type"],
                                                                                            data["breast density"],
                                                                                            data["subtlety"],
                                                                                            data["assessment"]
                                                                                            ): 
    print(".",end="")
    imgpath=imgpath.strip()
    roipath=roipath.strip()
    # obtiniendo informacion de la imagen
    patient_id,StudyInstanceUD1,SeriesInstanceUID1,filename=imgpath.rstrip().split("/")
    patient_id2,StudyInstanceUD2,SeriesInstanceUID2,filename2=roipath.rstrip().split("/")
 
    ruta_full=os.path.join(pathology,"full_mammo",patient_id,SeriesInstanceUID1)
    ruta_roi=os.path.join(pathology,"ground_truth",patient_id,SeriesInstanceUID2)
 

    # Si la ruta no existe, se la crea (es temporal)
    if not os.path.exists(ruta+ruta_full):
      os.makedirs(ruta+ruta_full)
    if not os.path.exists(rutadrive+ruta_full):
      os.makedirs(rutadrive+ruta_full)
    if not os.path.exists(rutadrive+ruta_roi):
      os.makedirs(rutadrive+ruta_roi)
 
    # descargando las imagenes del repositorio CBIS
    if not os.path.exists(os.path.join(ruta,ruta_full,"full_mammo.dcm")):
      descargadofull=tcia.get_image(SeriesInstanceUID1,ruta,"images.zip");
  
      #descomprimiendo las imagenes y renombrarlas a un nombre mas significativo
      if descargadofull:
        with zipfile.ZipFile(ruta+"images.zip", 'r') as zip_ref:
          member=zip_ref.namelist()[1]
          zip_ref.extract(member,ruta+ruta_full)
          os.rename(os.path.join(ruta,ruta_full,member),os.path.join(ruta,ruta_full,"full_mammo.dcm"))
      else:
        print(ruta_full+member)
    if not os.path.exists(os.path.join(os.path.join(ruta,ruta_roi,"roi_full_mammo.dcm"))):
      descargadoroicrop=tcia.get_image(SeriesInstanceUID2,ruta,"images.zip");
      if descargadoroicrop:
          with zipfile.ZipFile(ruta+"images.zip", 'r') as zip_ref:
            for member in zip_ref.namelist()[1:]:
              zip_ref.extract(member,ruta+ruta_roi)
              d = pydicom.read_file(os.path.join(ruta,ruta_roi,member))
              if d.SeriesDescription == "ROI mask images":
                full = pydicom.read_file(os.path.join(ruta,ruta_full,"full_mammo.dcm"))
                if d.Columns == full.Columns and d.Rows == full.Rows:
                  p = os.path.join(ruta_roi,"roi_full_mammo.dcm")
                  datos["roi_path"].append(p)
                  datos["image_path"].append(os.path.join(ruta_full,"full_mammo.dcm"))
                  datos["abnormality type"].append(abnormality)
                  datos["pathology"].append(pathology)
                  datos["patient_id"].append(p_id)
                  datos["patient_name"].append(patient_id)
                  datos["breast_density"].append(density)
                  datos["subtlety"].append(subtlety)
                  datos["assessment"].append(assessment)
                  os.rename(os.path.join(ruta,ruta_roi,member),ruta+p)
                break
    else:
      datos["roi_path"].append(os.path.join(ruta_roi,"roi_full_mammo.dcm"))
      datos["image_path"].append(os.path.join(ruta_full,"full_mammo.dcm"))
      datos["abnormality type"].append(abnormality)
      datos["pathology"].append(pathology)
      datos["patient_id"].append(p_id)
      datos["patient_name"].append(patient_id)
      datos["breast_density"].append(density)
      datos["subtlety"].append(subtlety)
      datos["assessment"].append(assessment)
    # else:
    #   descargadocrop=tcia.get_image(SeriesInstanceUID3,ruta,"images2.zip")
    #   if descargadoroicrop and descargadocrop:
    #     with zipfile.ZipFile(ruta+"images.zip", 'r') as zip_ref:
    #       member= zip_ref.namelist()[1]
    #       zip_ref.extract(member,ruta+ruta_roi)
    #       os.rename(os.path.join(ruta,ruta_roi,member),os.path.join(ruta,ruta_roi,"roi_full_mammo.dcm"))
    #       datos["roi_path"].append(os.path.join(ruta_roi,"roi_full_mammo.dcm"))
    #     with zipfile.ZipFile(ruta+"images2.zip", 'r') as zip_ref:
    #       member= zip_ref.namelist()[1]
    #       zip_ref.extract(member,ruta+ruta_roi)
    #       os.rename(os.path.join(ruta,ruta_roi,member),os.path.join(ruta,ruta_roi,"crop_mammo.dcm"))
    #       datos["cropped_path"].append(os.path.join(ruta_roi,"crop_mammo.dcm"))
  print()
  return datos

In [56]:
def do_convert_dicom_to_nii(datos):
  newdatos={}
  for tipo in datos:
    if tipo not in newdatos:
      newdatos[tipo]=[]
    for img in datos[tipo]:
      if ".dcm" not in str(img):
        newdatos[tipo].append(img)
      else:
        pathisto=rutadrive+img[:img.rfind("/")]+img[img.rfind("/"):-4]+".nii.gz"
        if not os.path.exists(pathisto):
          cmd=["dcm2niix","5","-b","n","-s","y","-z","y","-f",img[img.rfind("/")+1:-4],"-o",rutadrive+img[:img.rfind("/")+1],ruta+img]
          r=subprocess.run(cmd)
          if r.returncode!=0:
            print("Problema con "+img,r)
          # else: os.remove(ruta+img)
        newdatos[tipo].append(pathisto)
  return newdatos

Creando diccionarios para todo el grupo de anormalidad masa

In [57]:
# datos1=descargar_imagenes_imgs_2(tcia_client,df_masa_train2)
# dic_benigno_calc=do_convert_dicom_to_nii(datos1)

........................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................

In [73]:
# df_masa_train_nii=pd.DataFrame(dic_benigno_calc)
# df_masa_train_nii.shape

(1149, 9)

In [76]:
# df_masa_train_nii.drop(df_masa_train_nii[df_masa_train_nii.patient_name.isin(["Mass-Training_P_00034_RIGHT_MLO",
#                                                                               "Mass-Training_P_00370_RIGHT_MLO",
#                                                                               "Mass-Training_P_00406_RIGHT_MLO",
#                                                                               "Mass-Training_P_00637_RIGHT_CC",
#                                                                               "Mass-Training_P_00720_RIGHT_MLO",
#                                                                               "Mass-Training_P_01299_LEFT_MLO",
#                                                                               "Mass-Training_P_01371_RIGHT_MLO",
#                                                                               "Mass-Training_P_01532_LEFT_CC",
#                                                                               "Mass-Training_P_01560_RIGHT_MLO",
#                                                                               "Mass-Training_P_01611_RIGHT_MLO",
#                                                                               "Mass-Training_P_00637_RIGHT_CC"])].index,axis=0,inplace=True)
# df_masa_train_nii.reset_index(inplace=True,drop=True)
# print(df_masa_train_nii.shape)

(1139, 9)


In [77]:
# for i,j in zip(df_masa_train_nii.image_path,df_masa_train_nii.roi_path):
#   if not os.path.exists(i):
#     print(i)
#   if not os.path.exists(j):
#     print(j)

In [78]:
# df_masa_train_nii.to_csv(rutadrive+"mass_case_description_train_set_nii.csv",index=False)

In [79]:
# print(df_masa_train_nii["pathology"].value_counts())
# dff_2 = df_masa_train_nii[["pathology","patient_id"]]
# d=df_masa_train_nii.groupby(['patient_id','pathology']).count().reset_index()
# d["pathology"].value_counts()

MALIGNANT    615
BENIGN       524
Name: pathology, dtype: int64


MALIGNANT    335
BENIGN       278
Name: pathology, dtype: int64

In [ ]:
datos2=descargar_imagenes_imgs_2(tcia_client,df_masa_test2)
dic_masa_test=do_convert_dicom_to_nii(datos2)

....................................................................................................
Problema con MALIGNANT/full_mammo/Calc-Training_P_00034_RIGHT_MLO/1.3.6.1.4.1.9590.100.1.2.287974861112966343326395908123786241211/full_mammo.dcm CompletedProcess(args=['dcm2niix', '5', '-b', 'n', '-s', 'y', '-z', 'y', '-f', 'full_mammo', '-o', '/content/drive/My Drive/Materia_integradora/Datasets/CBIS-DDMS/MALIGNANT/full_mammo/Calc-Training_P_00034_RIGHT_MLO/1.3.6.1.4.1.9590.100.1.2.287974861112966343326395908123786241211/', '/content/Dataset/MALIGNANT/full_mammo/Calc-Training_P_00034_RIGHT_MLO/1.3.6.1.4.1.9590.100.1.2.287974861112966343326395908123786241211/full_mammo.dcm'], returncode=1)


Creando diccionarios para todo el grupo de anormalidad calcificacion

In [85]:
# datos3=descargar_imagenes_imgs_2(tcia_client,df_calc_train2)
# dic_calc_train=do_convert_dicom_to_nii(datos3)

........................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................

In [86]:
# df_calc_train_nii=pd.DataFrame(dic_calc_train)
# df_calc_train_nii.shape

(1071, 9)

In [87]:
# df_calc_train_nii.drop(df_calc_train_nii[df_calc_train_nii.patient_name.isin(["Calc-Training_P_00034_RIGHT_MLO",
#                                                                               "Calc-Training_P_00574_RIGHT_MLO",
#                                                                               "Calc-Training_P_00866_RIGHT_MLO",
#                                                                               "Calc-Training_P_01094_RIGHT_MLO",
#                                                                               "Calc-Training_P_01571_RIGHT_MLO",
#                                                                               "Calc-Training_P_01660_RIGHT_MLO",
#                                                                               "Calc-Training_P_02437_LEFT_MLO"])].index,axis=0,inplace=True)
# df_calc_train_nii.reset_index(inplace=True,drop=True)
# print(df_calc_train_nii.shape)

(1064, 9)


In [88]:
# for i,j in zip(df_calc_train_nii.image_path,df_calc_train_nii.roi_path):
#   if not os.path.exists(i):
#     print(i)
#   if not os.path.exists(j):
#     print(j)

In [89]:
# df_calc_train_nii.to_csv(rutadrive+"calc_case_description_train_set_nii.csv",index=False)

In [90]:
print(df_calc_train_nii["pathology"].value_counts())
dff_2 = df_calc_train_nii[["pathology","patient_id"]]
d=df_calc_train_nii.groupby(['patient_id','pathology']).count().reset_index()
d["pathology"].value_counts()

MALIGNANT    540
BENIGN       524
Name: pathology, dtype: int64


MALIGNANT    276
BENIGN       258
Name: pathology, dtype: int64

In [ ]:
datos4=descargar_imagenes_imgs_2(tcia_client,df_calc_test2)
dic_calc_test=do_convert_dicom_to_nii(datos4)

....................................................................................................
Problema con MALIGNANT/full_mammo/Mass-Training_P_00034_RIGHT_MLO/1.3.6.1.4.1.9590.100.1.2.369255064712196304011941917441138397637/full_mammo.dcm CompletedProcess(args=['dcm2niix', '5', '-b', 'n', '-s', 'y', '-z', 'y', '-f', 'full_mammo', '-o', '/content/drive/My Drive/Materia_integradora/Datasets/CBIS-DDMS/MALIGNANT/full_mammo/Mass-Training_P_00034_RIGHT_MLO/1.3.6.1.4.1.9590.100.1.2.369255064712196304011941917441138397637/', '/content/Dataset/MALIGNANT/full_mammo/Mass-Training_P_00034_RIGHT_MLO/1.3.6.1.4.1.9590.100.1.2.369255064712196304011941917441138397637/full_mammo.dcm'], returncode=1)


## csv Creados de Benigno y Maligno

In [ ]:
#CAMBIAR NOMBRES SEGUN LOS DF QUE SE USEN
nombresdf=["malignant_calc_case_description_train_set_new_100.csv",
           "malignant_mass_case_description_train_set_new_100.csv",
           "benign_calc_case_description_train_set_new_100.csv",
           "benign_mass_case_description_train_set_new_100.csv",
           "mass_test_nii.csv",
           "calc_test_nii.csv"]

In [ ]:
df_mlg_calc1=pd.DataFrame(dic_maligno_calc)
df_mlg_mass1=pd.DataFrame(dic_maligno_mass)
df_bng_calc1=pd.DataFrame(dic_benigno_calc)
df_bng_mass1=pd.DataFrame(dic_benigno_mass)

In [ ]:
print(df_mlg_calc1.shape)
print(df_mlg_mass1.shape)
print(df_bng_calc1.shape)
print(df_bng_mass1.shape)

(99, 4)
(98, 4)
(100, 4)
(96, 4)


In [ ]:
# df_mlg_mass1.drop(df_mlg_mass1[df_mlg_mass1.patient_id.isin(["Mass-Training_P_00034_RIGHT_MLO"])].index,axis=0,inplace=True)#.reset_index(inplace=True)
# df_mlg_mass1.reset_index(drop=True,inplace=True)
# df_mlg_mass1.shape

(97, 4)

In [ ]:
# df_mlg_calc1.drop(df_mlg_calc1[df_mlg_calc1.patient_id.isin(["Calc-Training_P_00034_RIGHT_MLO"])].index,axis=0,inplace=True)#.reset_index(inplace=True)
# df_mlg_calc1.reset_index(drop=True,inplace=True)
# df_mlg_calc1.shape

(98, 4)

In [ ]:
# df_bng_calc1.drop(df_bng_calc1[df_bng_calc1.patient_id.isin(["Calc-Training_P_01094_RIGHT_MLO","Calc-Training_P_01571_RIGHT_MLO","Calc-Training_P_01660_RIGHT_MLO"])].index,axis=0,inplace=True)#.reset_index(inplace=True)
# df_bng_calc1.reset_index(drop=True,inplace=True)
# df_bng_calc1.shape

(261, 5)

In [ ]:
# df_bng_mass1.drop(df_bng_mass1[df_bng_mass1.patient_id.isin(["Mass-Training_P_01299_LEFT_MLO","Mass-Training_P_01486_RIGHT_MLO","Mass-Training_P_01532_LEFT_CC","Mass-Training_P_01560_RIGHT_MLO","Mass-Training_P_01611_RIGHT_MLO"])].index,axis=0,inplace=True)#.reset_index(inplace=True)
# df_bng_mass1.reset_index(drop=True,inplace=True)
# df_bng_mass1.shape

(284, 5)

In [ ]:
# df_mlg_calc=pd.concat([df_mlg_calc,df_mlg_calc1],ignore_index=True)
# df_mlg_calc.shape

(542, 5)

In [ ]:
# df_mlg_mass=pd.concat([df_mlg_mass,df_mlg_mass1],ignore_index=True)
# df_mlg_mass.shape

(630, 5)

In [ ]:
# df_bng_calc=pd.concat([df_bng_calc,df_bng_calc1],ignore_index=True)
# df_bng_calc.shape

(524, 5)

In [ ]:
# df_bng_mass=pd.concat([df_bng_mass,df_bng_mass1],ignore_index=True)
# df_bng_mass.shape

(572, 5)

In [ ]:
# df_mlg_calc1.to_csv(rutadrive+nombresdf[0])
# df_mlg_mass1.to_csv(rutadrive+nombresdf[1])
# df_bng_calc1.to_csv(rutadrive+nombresdf[2])
# df_bng_mass1.to_csv(rutadrive+nombresdf[3])
# df_mass_test.to_csv(rutadrive+nombresdf[4])
# df_calci_test.to_csv(rutadrive+nombresdf[5])

In [ ]:
# df_mlg_calc=pd.read_csv(rutadrive+nombresdf[0],index_col=0)
# df_mlg_mass=pd.read_csv(rutadrive+nombresdf[1],index_col=0)
# df_bng_calc=pd.read_csv(rutadrive+nombresdf[2],index_col=0)
# df_bng_mass=pd.read_csv(rutadrive+nombresdf[3],index_col=0)
# df_mass_test=pd.read_csv(rutadrive+nombresdf[4],index_col=0)
# df_calci_test=pd.read_csv(rutadrive+nombresdf[5],index_col=0)

In [ ]:
print(df_mlg_calc.shape)
print(df_mlg_mass.shape)
print(df_bng_calc.shape)
print(df_bng_mass.shape)
print(df_mass_test.shape)
print(df_calci_test.shape)

(541, 9)
(630, 9)
(524, 9)
(572, 9)
(338, 6)
(254, 6)


## Obteniendo imagenes normales

In [ ]:
from ftplib import FTP
import os
import urllib.request as req
import shutil
from contextlib import closing
 
def dowload_normales():
  n=0
  host="figment.csee.usf.edu"
  folder = ruta + 'NORMAL/'
  endrive = rutadrive+'NORMAL/'
  if not os.path.exists(folder):
    os.makedirs(folder)
  lista=[]
  with FTP(host,timeout=30) as ftp:
      if not os.path.exists(folder):
          os.makedirs(folder)
      if not os.path.exists(endrive):
          os.makedirs(endrive)
      # Login to FTP Server
      ftp.login()
      normals="pub/DDSM/cases/normals"
      # Get the list of files in the server
      lista_normales = ftp.nlst(normals)
      for dir in lista_normales:
        # Change the working directory
        ftp.cwd("/"+dir+"/")
        lista_casos=ftp.nlst()
        for caso in lista_casos: 
          if n<150+238:
            n+=1
            continue
          print(dir,caso)
 
          lista.append("NORMAL/"+caso)
          if os.path.exists(endrive+caso+"/"):
            if len(os.listdir(endrive+caso+"/"))>=5:
              continue
          else: 
            os.makedirs(endrive+caso+"/")
 
          if os.path.exists(folder+caso+"/"):
            if len(os.listdir(folder+caso+"/"))>=5:
              continue
          else: 
            os.makedirs(folder+caso+"/")
          
          ftp.cwd("/"+dir+"/"+caso+"/")   
          for filename in ftp.nlst():
            if ".16_PGM" in filename:
              continue
            # Create the URL from where file has to be downloaded
            file = 'ftp://{}/{}/{}/{}'.format(host,dir,caso,filename)
            # Create the file name
            download_file_name = folder + caso+"/"+filename
            with closing(req.urlopen(file)) as r:
                with open(download_file_name, 'wb') as f:
                    shutil.copyfileobj(r, f)
  return lista

In [ ]:
# prueba=dowload_normales()

In [ ]:
def split_filename(filepath):
  path = os.path.dirname(filepath)
  filename = os.path.basename(filepath)
  base, ext = os.path.splitext(filename)
  return path, base, ext

In [ ]:
#IMAGENES TIFF A DICOM
def convert_tiff_dicom(ics_dict,tiff,force=False):
  """
  :param ics_dict: dict with infomation of the image
  :param tif: path to tif image
  :param force: if True, force the creation of the file
  :return: converted file path
  """
  path,base,ext=split_filename(tiff)
  dicom_filename=os.path.join(base+".dcm")
 
  if os.path.exists(path+"/"+dicom_filename) and not force:
    return (base,dicom_filename)
  _,gg=base.split(".")
  # Create the metadata for the dataset
  file_meta = Dataset()
  #file_meta.TransferSyntaxUID = ImplicitVRLittleEndian
 
  # Create some temporary filenames
  suffix = '.dcm'
  filename_little_endian = tempfile.NamedTemporaryFile(suffix=suffix).name
 
  # CONFIGURACION DE IMAGEN DICOM BASE CBIS DDSM
  file_meta.MediaStorageSOPClassUID = 'Secondary Capture Image Storage'
  file_meta.MediaStorageSOPInstanceUID = '1.3.6.1.4.1.9590.100.1.2.289923739312470966435676008311959891294'
  file_meta.ImplementationClassUID = '1.2.40.0.13.1.1.1'
  
  # Create the dataset
  ds = FileDataset(filename_little_endian, {}, file_meta=file_meta, preamble=b'\x00'*128) # pre-existing dicom file
  ds.is_little_endian = True
  ds.is_implicit_VR = True
 
  ds.PatientName = ics_dict["patient_id"]
  ds.PatientID = ics_dict["patient_id"]
  im_frame = Image.open(tiff)
  np_frame = np.array(im_frame.getdata(),dtype=np.uint16)
  ds.Rows = ics_dict[gg]["height"]
  ds.Columns = ics_dict[gg]["width"]
  ds.PhotometricInterpretation = "MONOCHROME2"
  ds.BitsAllocated = ics_dict[gg]["bpp"]
  ds.BitsStored = ics_dict[gg]["bpp"]
  ds.SmallestImagePixelValue = 0
  ds.LargestImagePixelValue  = 65535
  ds.HighBit = 15
  ds.PixelRepresentation = 0    #fijo, todos ejemplos usan 0
  ds.PixelData = np_frame.tobytes()
  ds.save_as(path+"/"+dicom_filename)
  return (base,dicom_filename)

In [ ]:
def convert_LJPEG_dcm(dirs):
  datos={"patient_id":[],"image_path":[]}
  for dir in dirs:
    try:
      ics = glob.glob(ruta+dir + '/*.ics')[0]
    except:
      continue
    ics_dict=ddsm.get_ics_info(ics)
   
    for f in os.listdir(ruta+dir):
      
      if ".LJPEG" not in f:
        continue
      
      normal = ddsm.ddsm_abnormality(os.path.join(ruta,dir,f),ics_dict)
      tif_file=normal.save_image(od_correct=True)
      patient_id,dcm_file=convert_tiff_dicom(ics_dict,tif_file)      
      datos["patient_id"].append(patient_id)
      datos["image_path"].append(os.path.join(dir,dcm_file))
      os.remove(os.path.join(ruta,dir,f))
  return datos

In [ ]:
# dcms=convert_LJPEG_dcm(prueba)

In [ ]:
#normales=do_convert_dicom_to_nii(dcms)

In [ ]:
# df_mlg_calc["pathology"]="MALIGNANT"
# df_mlg_calc['id']=df_mlg_calc['patient_id'].apply(lambda x: x.split("_",1)[1][:7])
# df_mlg_calc[['left or right breast',"image view"]]=df_mlg_calc['patient_id'].str.split("_",1,True)[1].str[8:].str.split("_",expand=True)

# df_mlg_mass["pathology"]="MALIGNANT"
# df_mlg_mass['id']=df_mlg_mass['patient_id'].apply(lambda x: x.split("_",1)[1][:7])
# df_mlg_mass[['left or right breast',"image view"]]=df_mlg_mass['patient_id'].str.split("_",1,True)[1].str[8:].str.split("_",expand=True)

# df_bng_calc["pathology"]="BENIGN"
# df_bng_calc['id']=df_bng_calc['patient_id'].apply(lambda x: x.split("_",1)[1][:7])
# df_bng_calc[['left or right breast',"image view"]]=df_bng_calc['patient_id'].str.split("_",1,True)[1].str[8:].str.split("_",expand=True)

# df_bng_mass["pathology"]="BENIGN"
# df_bng_mass['id']=df_bng_mass['patient_id'].apply(lambda x: x.split("_",1)[1][:7])
# df_bng_mass[['left or right breast',"image view"]]=df_bng_mass['patient_id'].str.split("_",1,True)[1].str[8:].str.split("_",expand=True)

# df_calci_test['id']=df_calci_test['patient_id'].apply(lambda x: x.split("_",1)[1][:7])
# df_mass_test['id']=df_mass_test['patient_id'].apply(lambda x: x.split("_",1)[1][:7])

In [ ]:
df=pd.concat([df_mlg_calc,df_bng_calc],ignore_index=True)
print(df["pathology"].value_counts())
dff_2 = df[["pathology","id"]].groupby(['id','pathology']).count().reset_index()
dff_2["pathology"].value_counts()

MALIGNANT    541
BENIGN       524
Name: pathology, dtype: int64


MALIGNANT    276
BENIGN       258
Name: pathology, dtype: int64

In [ ]:
df=pd.concat([df_mlg_mass,df_bng_mass],ignore_index=True)
print(df["pathology"].value_counts())
dff_2 = df[["pathology","id"]].groupby(['id','pathology']).count().reset_index()
dff_2["pathology"].value_counts()

MALIGNANT    630
BENIGN       572
Name: pathology, dtype: int64


MALIGNANT    344
BENIGN       303
Name: pathology, dtype: int64

In [ ]:
print(df_calci_test["pathology"].value_counts())
dff_2 = df_calci_test[["pathology","id"]].groupby(['id','pathology']).count().reset_index()
dff_2["pathology"].value_counts()

MALIGNANT    128
BENIGN       126
Name: pathology, dtype: int64


BENIGN       69
MALIGNANT    67
Name: pathology, dtype: int64

In [ ]:
print(df_mass_test["pathology"].value_counts())
dff_2 = df_mass_test[["pathology","id"]].groupby(['id','pathology']).count().reset_index()
dff_2["pathology"].value_counts()

BENIGN       192
MALIGNANT    146
Name: pathology, dtype: int64


BENIGN       100
MALIGNANT     85
Name: pathology, dtype: int64